In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
from sentences import *
from DCS import *

from MatDB import *
import word_definite as WD
from heap_n_PrimMST import *
from nnet import *

In [3]:
loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'), encoding=u'utf-8')
loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'), encoding=u'utf-8')

In [4]:
def GetTrainingKit(sentenceObj, dcsObj):
    nodelist = GetNodes(sentenceObj)
    
    # Nodelist with only the correct_nodes
    nodelist2 = GetNodes(sentenceObj)
    nodelist2_to_correct_mapping = {}
    nodelist_correct = []
    search_key = 0
    first_key = 0
    for chunk_id in range(len(dcsObj.lemmas)):
        while nodelist2[first_key].chunk_id != chunk_id:
            first_key += 1
        for j in range(len(dcsObj.lemmas[chunk_id])):
            search_key = first_key
            while (nodelist2[search_key].lemma != rom_slp(dcsObj.lemmas[chunk_id][j])) or (nodelist2[search_key].cng != dcsObj.cng[chunk_id][j]):
                search_key += 1
                if search_key >= len(nodelist2) or nodelist2[search_key].chunk_id > chunk_id:
                    break
    #         print((rom_slp(dcsObj.lemmas[chunk_id][j]), dcsObj.cng[chunk_id][j]))
    #         print(nodelist[search_key])
            nodelist2_to_correct_mapping[len(nodelist_correct)] = search_key
            nodelist_correct.append(nodelist2[search_key])
    return (nodelist, nodelist_correct, nodelist2_to_correct_mapping)
    

def GetGraph(nodelist, neuralnet):
    if not neuralnet.outer_relu:
        conflicts_Dict = Get_Conflicts(nodelist)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

        (WScalarMat, SigmoidGateOutput) = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        return (conflicts_Dict, featVMat, WScalarMat, SigmoidGateOutput)
    else:
        conflicts_Dict = Get_Conflicts(nodelist)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

        WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        return (conflicts_Dict, featVMat, WScalarMat)
def Test(sentenceObj, dcsObj):
    try:
        (_, _, _) = GetTrainingKit(sentenceObj, dcsObj)
        return True
        # nodelist = GetNodes(sentenceObj)
    except IndexError:
        # print('\x1b[31mError with {} \x1b[0m'.format(sentenceObj.sent_id))
        return False

In [5]:
def main(loaded_SKT, loaded_DCS, trainSize = 100, testSize = 100):
    test_set = set()
    train_set = set()
    
    _skt_file_list = list(loaded_SKT.keys())
    perm = np.random.permutation(len(loaded_SKT))

    print('Get Train Permutation')

    fn = 0
    while len(train_set) < trainSize:
        sentenceObj = loaded_SKT[_skt_file_list[fn]]
        dcsObj = loaded_DCS[_skt_file_list[fn]]
        if Test(sentenceObj, dcsObj):
            train_set.add(_skt_file_list[fn])
            
        fn += 1
    
    print('Get Test Permutation')

    # Run few times on same set of files   
    while len(test_set) < testSize:
        sentenceObj = loaded_SKT[_skt_file_list[fn]]
        dcsObj = loaded_DCS[_skt_file_list[fn]]
        if Test(sentenceObj, dcsObj):
            test_set.add(_skt_file_list[fn])
        fn += 1
    print('Train set size: ', len(train_set))
    print('Test set size: ', len(test_set))
    if len(train_set) == len(train_set - test_set):
        print('Both set\'s are disjoint...')
    else:
        print('Both set\'s are not disjoint...')
    return (list(train_set), list(test_set))

In [7]:
TrainFiles, TestFiles = main(loaded_SKT, loaded_DCS, trainSize=6000, testSize=3000)

Get Train Permutation
Get Test Permutation
Train set size:  6000
Test set size:  3000
Both set's are disjoint...


In [8]:
pickle.dump({'TestFiles': TestFiles, 'TrainFiles': TrainFiles}, open('../SmallDataset_6K_3K.p', 'wb'))

### Split folder of bz2 compressed files into test and train data sets

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## bUILT-iN pACKAGES
import sys, os, time, bz2, zlib, pickle, math, json
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
%matplotlib inline
from IPython.display import display

## lAST sUMMER
from romtoslp import *
from sentences import *
from DCS import *
import MatDB

In [3]:
all_filelist = os.listdir('../NewData/skt_dcs_DS.bz2_10K/')
all_filelist.sort()
dsbz2_filelist = []
for fn in all_filelist:
    if '.ds.bz2' in fn:
        dsbz2_filelist.append(fn)
        
# dsbz2_filelist

In [4]:
file_perm = np.random.permutation(len(dsbz2_filelist))

n_trainset = int(file_perm.shape[0]/2)
# n_trainset = 400

file_perm_train = file_perm[:n_trainset].astype(np.int32)
file_perm_test = file_perm[n_trainset:]

In [5]:
Train_bz2Files = []
Test_bz2Files = []
for x in file_perm_train:
    Train_bz2Files.append(dsbz2_filelist[x])
for x in file_perm_test:
    Test_bz2Files.append(dsbz2_filelist[x])
print('Training set size: ', len(Train_bz2Files))
print('Test set size: ', len(Test_bz2Files))
print('Intersection: ', len(set(Train_bz2Files) - set(Test_bz2Files)) - len(Train_bz2Files))

Training set size:  4046
Test set size:  4047
Intersection:  0


In [6]:
pickle.dump({
        'TrainFiles': Train_bz2Files,
        'TestFiles': Test_bz2Files
    }, open('../bz2Dataset_10K.p', 'wb'))

### Read Held out dataset

In [13]:
import csv, os

In [5]:
baseline_filelist = []
with open('inputs/Baseline4_advSample.csv') as f:
    baseline_reader = csv.reader(f)
    for line in baseline_reader:
        baseline_filelist.append(line[1])

In [10]:
heldout_names = [f.replace('.p', '.ds.bz2') for f in baseline_filelist]

In [17]:
c = 0
for f in heldout_names:
    c += os.path.isfile('../NewData/skt_dcs_DS.bz2_10K/' + f)

In [18]:
c

957